In [49]:
library(hipathia)
library(dplyr)


Attaching package: ‘dplyr’


The following object is masked from ‘package:Biobase’:

    combine


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following object is masked from ‘package:matrixStats’:

    count


The following objects are masked from ‘package:dbplyr’:

    ident, sql


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:igraph’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, se

In [2]:
### genes that belong to circuits

In [3]:
## dataframe for each gene belong to pathway.

pathways <- load_pathways("hsa")
names_circs <- names(pathways$pathigraphs)
# extrac genes from a pathways
f_extract_genes <- function(j){

	genes <- unique(na.omit(unlist(V(getElement(get_element, j))$genesList)))
	genes_clean <- genes[genes != "/" & genes != "NA"]
	return(genes_clean)
}

# join every genes of every path
genes_com <- c()
circs_com <- c()
df <- c()

for (i in 1:length(names_circs)){
	x <- names_circs[i]
	get_element <- getElement(pathways$pathigraphs,x)$effector.subgraphs

	list_path_genes <- sapply(names(get_element), f_extract_genes)

	genes <- paste(do.call(c, list_path_genes)) 
	circs <- paste(do.call(c,sapply(1:length(list_path_genes), FUN=function(z) rep(names(list_path_genes)[z], length(list_path_genes[[z]])))))	
	genes_com <- c(genes_com, genes)
	circs_com <- c(circs_com, circs)	
}


df_circ_genes <- data.frame(circs = circs_com, genes = genes_com)


physiological_paths <- read.table("/mnt/lustre/scratch/CBRA/research/projects/rrdd_mm/data/physiological_paths.tsv", sep="\t") 
df_circ_physi_genes <- df_circ_genes[unlist(lapply(physiological_paths$V2, FUN=function(x) grep(x, df_circ_genes$circs))),]
df_circ_physi_genes$circ_name <- get_path_names(pathways, df_circ_physi_genes$circs)

Loaded 146 pathways



In [4]:
saveRDS(df_circ_physi_genes, file="/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/circ_physi_genes.rds")

In [5]:
### drugs and their target of drugbank

In [9]:
drug_bank <- read.table("/mnt/lustre/scratch/CBRA/data/DRUGBANK/drugbank_drug-bindings_v5.1.8.tsv", sep="\t", header=T)
drug_bank_sel <- unique(drug_bank[,c("drug_name","entrez_id","actions")])
drug_bank_sel$drug_name <- tolower(drug_bank_sel$drug_name)

In [ ]:
### projects of tcga selected

In [27]:
proj_tcga_sel <- c("tcga_brca", "tcga_coad","tcga_esca","tcga_lgg","tcga_luad","tcga_lusc","tcga_paad","tcga_skcm","tcga_stad")
lapply(proj_tcga_sel, FUN=function(x) dir.create(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_2_pat_drug_db_circ/",x, sep="")))

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

In [ ]:
### extract target of drug used in patients

In [13]:
funct_pat_drug_db_circ <- function(x){
    file <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_1_correc_drug/",x,"/pat_drug_corr_",x,".rds",sep="")
    pat_drug_corr <- readRDS(file)

    pat_drug_db <- drug_bank_sel[drug_bank_sel$drug_name %in% unique(pat_drug_corr$drug_name),]
    pat_drug_db_circ <- merge(pat_drug_db, df_circ_physi_genes, by.x="entrez_id",by.y="genes")
    fils <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_2_pat_drug_db_circ/",x,"/pat_drug_db_circ_",x,".rds",sep="")
    saveRDS(pat_drug_db_circ, file=fils)
}

In [14]:
lapply(proj_tcga_sel, FUN=function(x) funct_pat_drug_db_circ(x))

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

In [40]:
p

entrez_id,drug_name,actions,circs,circ_name
<int>,<chr>,<chr>,<chr>,<chr>
2,cisplatin,NA,P-hsa04610-57,Complement and coagulation cascades: PLAUR
2,cisplatin,NA,P-hsa04610-17,Complement and coagulation cascades: PLG
2,cisplatin,NA,P-hsa04610-58,Complement and coagulation cascades: F2RL2
2,cisplatin,NA,P-hsa04610-11,Complement and coagulation cascades: F2R
2,cisplatin,NA,P-hsa04610-52,Complement and coagulation cascades: CPB2
2,cisplatin,NA,P-hsa04610-7,Complement and coagulation cascades: Fibrin
291,clodronic acid,inhibitor,P-hsa04022-71,cGMP-PKG signaling pathway: PPIF
292,clodronic acid,inhibitor,P-hsa04022-71,cGMP-PKG signaling pathway: PPIF
293,clodronic acid,inhibitor,P-hsa04022-71,cGMP-PKG signaling pathway: PPIF


In [43]:
proj_tcga_sel <- c("tcga_brca", "tcga_coad","tcga_esca","tcga_lgg","tcga_luad","tcga_lusc","tcga_paad","tcga_skcm","tcga_stad")

get <- function(x){
    fils <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_2_pat_drug_db_circ/",x,"/pat_drug_db_circ_",x,".rds",sep="")
    file <- readRDS(fils)
    file$project <- rep(x, length(file$actions))
    return(file)
}

In [59]:
get_drug_actions <- do.call(rbind,lapply(proj_tcga_sel, FUN=function(x) get(x)))

In [64]:
unique(data.frame(get_drug_actions %>% filter(actions == "inducer,activator" | actions == "stimulator"))[,c("entrez_id","project")])

,entrez_id,project
,<int>,<chr>
1,5564,tcga_brca
76,1441,tcga_coad
